In [31]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import functions as fun
from linearmodels.iv import IV2SLS

pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

In [32]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,1.037e-02,Aiways,U5,400,2.846e+05,201,34,SUV,C,CN,257,1.0
9,1,2022,5.976e-03,Aiways,U5,400,3.137e+05,201,34,SUV,C,CN,183,1.0
10,1,2023,2.860e-03,Aiways,U5,400,2.645e+05,201,34,SUV,C,CN,177,1.0
21,2,2023,4.848e-05,Aiways,U6,405,3.606e+05,214,34,SUV,C,CN,3,1.0
28,3,2019,4.063e-02,Audi,e-tron,375,9.797e+05,402,17,SUV,F,DE,222,1.0
29,3,2020,3.468e-02,Audi,e-tron,375,8.901e+05,402,17,SUV,F,DE,491,1.0
30,3,2021,1.049e-02,Audi,e-tron,375,8.000e+05,402,17,SUV,F,DE,260,1.0
31,3,2022,1.757e-02,Audi,e-tron,375,7.897e+05,402,17,SUV,F,DE,538,1.0
32,3,2023,1.099e-03,Audi,e-tron,375,6.730e+05,402,17,SUV,F,DE,68,1.0
41,4,2021,3.391e-03,Audi,e-tron GT,472,1.279e+06,522,17,Sedan,F,DE,84,1.0


In [33]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,1.037e-02,Aiways,U5,400,2.846e+05,201,34,SUV,C,CN,257,1.0
1,1,2022,5.976e-03,Aiways,U5,400,3.137e+05,201,34,SUV,C,CN,183,1.0
2,1,2023,2.860e-03,Aiways,U5,400,2.645e+05,201,34,SUV,C,CN,177,1.0
3,2,2023,4.848e-05,Aiways,U6,405,3.606e+05,214,34,SUV,C,CN,3,1.0
4,3,2019,4.063e-02,Audi,e-tron,375,9.797e+05,402,17,SUV,F,DE,222,1.0
5,3,2020,3.468e-02,Audi,e-tron,375,8.901e+05,402,17,SUV,F,DE,491,1.0
6,3,2021,1.049e-02,Audi,e-tron,375,8.000e+05,402,17,SUV,F,DE,260,1.0
7,3,2022,1.757e-02,Audi,e-tron,375,7.897e+05,402,17,SUV,F,DE,538,1.0
8,3,2023,1.099e-03,Audi,e-tron,375,6.730e+05,402,17,SUV,F,DE,68,1.0
9,4,2021,3.391e-03,Audi,e-tron GT,472,1.279e+06,522,17,Sedan,F,DE,84,1.0


In [34]:
fun.BLP(data, 'Range')
fun.BLP(data, 'HP')
fun.BLP(data, 'Chargetime')

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,Range_BLP,HP_BLP,Chargetime_BLP
0,1,2021,1.037e-02,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0,21513,13583,1861
1,1,2022,5.976e-03,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0,32180,20339,2664
2,1,2023,2.860e-03,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0,41774,26403,3241
3,2,2023,4.848e-05,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0,41769,26390,3241
4,3,2019,4.063e-02,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0,5809,3901,570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,189,2023,6.464e-05,Volvo,EX30,475,368245.000,268,28,SUV,B,SE,4,1.0,41699,26336,3247
330,190,2021,1.465e-02,Volvo,XC40,457,462060.600,402,28,SUV,C,SE,363,1.0,21456,13382,1867
331,190,2022,3.331e-02,Volvo,XC40,457,416263.400,402,28,SUV,C,SE,1020,1.0,32123,20138,2670
332,190,2023,3.175e-02,Volvo,XC40,457,439266.600,402,28,SUV,C,SE,1965,1.0,41717,26202,3247


In [35]:
product_data = data.rename(columns={
    'Year': 'market_ids',
    'Model': 'product_ids',
    'Market_share': 'shares',
    'Price': 'prices',
    'Manufacturer': 'firm_ids',
    'Range_BLP': 'demand_instruments0',
    'HP_BLP': 'demand_instruments1',
    'Chargetime_BLP': 'demand_instruments2',
})

In [36]:
#Scale for better intepretation
product_data['prices'] = product_data['prices']/10_000 #(Change in ms(%) for change in pris in 10.000)
product_data['HP'] = product_data['HP']/10           #(Change in ms(%) for change in HP in 10)
product_data['Range'] = product_data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [37]:
# Creating dummy for china
product_data['China'] = (product_data['Country'] == 'CN').astype(int)

# Outside share

In [38]:
product_data.loc[product_data['market_ids'] == 2013, 'shares'] = product_data.loc[product_data['market_ids'] == 2013, 'Sales'] / 180632
product_data.loc[product_data['market_ids'] == 2014, 'shares'] = product_data.loc[product_data['market_ids'] == 2014, 'Sales'] / 188406
product_data.loc[product_data['market_ids'] == 2015, 'shares'] = product_data.loc[product_data['market_ids'] == 2015, 'Sales'] / 206653
product_data.loc[product_data['market_ids'] == 2016, 'shares'] = product_data.loc[product_data['market_ids'] == 2016, 'Sales'] / 222471
product_data.loc[product_data['market_ids'] == 2017, 'shares'] = product_data.loc[product_data['market_ids'] == 2017, 'Sales'] / 221471
product_data.loc[product_data['market_ids'] == 2018, 'shares'] = product_data.loc[product_data['market_ids'] == 2018, 'Sales'] / 252328
product_data.loc[product_data['market_ids'] == 2019, 'shares'] = product_data.loc[product_data['market_ids'] == 2019, 'Sales'] / 258727
product_data.loc[product_data['market_ids'] == 2020, 'shares'] = product_data.loc[product_data['market_ids'] == 2020, 'Sales'] / 230060
product_data.loc[product_data['market_ids'] == 2021, 'shares'] = product_data.loc[product_data['market_ids'] == 2021, 'Sales'] / 222210
product_data.loc[product_data['market_ids'] == 2022, 'shares'] = product_data.loc[product_data['market_ids'] == 2022, 'Sales'] / 181030
product_data.loc[product_data['market_ids'] == 2023, 'shares'] = product_data.loc[product_data['market_ids'] == 2023, 'Sales'] / 203690

In [39]:
product_data['outside_share'] = 1 - product_data.groupby('market_ids')['shares'].transform('sum')
product_data[['shares', 'outside_share']].describe()

,shares,outside_share
count,3.340e+02,334.000
mean,2.128e-03,0.841
std,5.618e-03,0.109
min,4.347e-06,0.696
25%,1.227e-04,0.696
50%,7.149e-04,0.831
75%,2.161e-03,0.938
max,8.814e-02,0.998


# Demand

In [40]:
product_data['logit_delta'] = np.log(product_data['shares'] / product_data['outside_share'])
formula = 'logit_delta ~ 1 + [prices ~ demand_instruments0 + demand_instruments1 + demand_instruments2] + Range + HP + Chargetime + China'
IV = IV2SLS.from_formula(formula, product_data).fit(cov_type='robust')
IV.first_stage

,prices
R-squared,0.5822
Partial R-squared,0.0261
Shea's R-squared,0.0261
Partial F-statistic,17.145
P-value (Partial F-stat),0.0007
Partial F-stat Distn,chi2(3)
==========================,===========
Intercept,-1.3281
,(-0.1735)
Range,0.1875


In [41]:
IV.summary#.tables[1]

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:            logit_delta   R-squared:                      0.1614
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1486
No. Observations:                 334   F-statistic:                    101.16
Date:                Mon, Jun 03 2024   P-value (F-stat)                0.0000
Time:                        22:36:38   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -9.0442     0.7141    -12.665     0.0000     -10.444     -7.6446
Range          0.0979     0.0207     4.7296     0.0000      0.0573      0.1384
HP            -0.0687     0.0370    -1.8581     0.0632     -0.1412      0.0038
Chargetime    -0.0313     0.0117    -2.6686     0.0076     -0.0543     -0.0083
China         -0.5112     0.4488    -1.1391     0.2547     -1.3908      0.3684
prices         0.0121     0.0324     0.3734     0.7089     -0.0514      0.0757
==============================================================================

Endogenous: prices
Instruments: demand_instruments0, demand_instruments1, demand_instruments2
Robust Covariance (Heteroskedastic)
Debiased: False
"""

In [42]:
BLP_problem = pyblp.Problem(pyblp.Formulation('1 + prices + Range + HP + Chargetime + China'), product_data)
BLP_results = BLP_problem.solve(method='1s')

In [43]:
data2023 = product_data[product_data['market_ids'] == 2023]
ols_elasticities = BLP_results.compute_elasticities(market_id=2023) # name = 'Range' for elasticities of Range
pd.DataFrame(ols_elasticities, index = data2023['product_ids'], columns=data2023['product_ids'])

product_ids,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,iX,iX1,iX3,Atto 3,Dolphin,Han,Seal,Tang,Berlingo,C4,ë-C4 X,e-SpaceTourer,Jumpy,Born,Spring,...,Taycan,Megane,Zoe,Enyaq iV,Fortwo,Korando,Solterra,Model 3,Model S,Model X,Model Y,bZ4X,Proace City Verso,Proace Verso,Golf,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
product_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
U5,3.200e-01,-6.430e-06,-2.720e-04,-1.667e-04,-0.009,-0.002,-1.883e-06,-0.006,-0.002,-4.656e-04,-0.002,-0.003,-0.003,-4.090e-04,-2.814e-04,-1.053e-05,-6.098e-05,-1.665e-05,-5.051e-05,-3.676e-04,-3.206e-04,-1.100e-04,-1.701e-05,-0.003,-8.506e-05,...,-0.001,-0.002,-4.530e-05,-0.008,-3.753e-06,-6.783e-06,-1.124e-04,-0.009,-7.528e-04,-4.429e-04,-0.043,-0.002,-1.754e-05,-4.941e-05,-1.857e-06,-0.002,-0.006,-0.002,-5.787e-04,-0.004,-8.210e-05,-0.001,-8.755e-06,-0.005,-2.400e-05
U6,-2.783e-04,4.366e-01,-2.720e-04,-1.667e-04,-0.009,-0.002,-1.883e-06,-0.006,-0.002,-4.656e-04,-0.002,-0.003,-0.003,-4.090e-04,-2.814e-04,-1.053e-05,-6.098e-05,-1.665e-05,-5.051e-05,-3.676e-04,-3.206e-04,-1.100e-04,-1.701e-05,-0.003,-8.506e-05,...,-0.001,-0.002,-4.530e-05,-0.008,-3.753e-06,-6.783e-06,-1.124e-04,-0.009,-7.528e-04,-4.429e-04,-0.043,-0.002,-1.754e-05,-4.941e-05,-1.857e-06,-0.002,-0.006,-0.002,-5.787e-04,-0.004,-8.210e-05,-0.001,-8.755e-06,-0.005,-2.400e-05
e-tron,-2.783e-04,-6.430e-06,8.145e-01,-1.667e-04,-0.009,-0.002,-1.883e-06,-0.006,-0.002,-4.656e-04,-0.002,-0.003,-0.003,-4.090e-04,-2.814e-04,-1.053e-05,-6.098e-05,-1.665e-05,-5.051e-05,-3.676e-04,-3.206e-04,-1.100e-04,-1.701e-05,-0.003,-8.506e-05,...,-0.001,-0.002,-4.530e-05,-0.008,-3.753e-06,-6.783e-06,-1.124e-04,-0.009,-7.528e-04,-4.429e-04,-0.043,-0.002,-1.754e-05,-4.941e-05,-1.857e-06,-0.002,-0.006,-0.002,-5.787e-04,-0.004,-8.210e-05,-0.001,-8.755e-06,-0.005,-2.400e-05
e-tron GT,-2.783e-04,-6.430e-06,-2.720e-04,1.476e+00,-0.009,-0.002,-1.883e-06,-0.006,-0.002,-4.656e-04,-0.002,-0.003,-0.003,-4.090e-04,-2.814e-04,-1.053e-05,-6.098e-05,-1.665e-05,-5.051e-05,-3.676e-04,-3.206e-04,-1.100e-04,-1.701e-05,-0.003,-8.506e-05,...,-0.001,-0.002,-4.530e-05,-0.008,-3.753e-06,-6.783e-06,-1.124e-04,-0.009,-7.528e-04,-4.429e-04,-0.043,-0.002,-1.754e-05,-4.941e-05,-1.857e-06,-0.002,-0.006,-0.002,-5.787e-04,-0.004,-8.210e-05,-0.001,-8.755e-06,-0.005,-2.400e-05
Q4 e-tron,-2.783e-04,-6.430e-06,-2.720e-04,-1.667e-04,0.788,-0.002,-1.883e-06,-0.006,-0.002,-4.656e-04,-0.002,-0.003,-0.003,-4.090e-04,-2.814e-04,-1.053e-05,-6.098e-05,-1.665e-05,-5.051e-05,-3.676e-04,-3.206e-04,-1.100e-04,-1.701e-05,-0.003,-8.506e-05,...,-0.001,-0.002,-4.530e-05,-0.008,-3.753e-06,-6.783e-06,-1.124e-04,-0.009,-7.528e-04,-4.429e-04,-0.043,-0.002,-1.754e-05,-4.941e-05,-1.857e-06,-0.002,-0.006,-0.002,-5.787e-04,-0.004,-8.210e-05,-0.001,-8.755e-06,-0.005,-2.400e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,-2.783e-04,-6.430e-06,-2.720e-04,-1.667e-04,-0.009,-0.002,-1.883e-06,-0.006,-0.002,-4.656e-04,-0.002,-0.003,-0.003,-4.090e-04,-2.814e-04,-1.053e-05,-6.098e-05,-1.665e-05,-5.051e-05,-3.676e-04,-3.206e-04,-1.100e-04,-1.701e-05,-0.003,-8.506e-05,...,-0.001,-0.002,-4.530e-05,-0.008,-3.753e-06,-6.783e-06,-1.124e-04,-0.009,-7.528e-04,-4.429e-04,-0.043,-0.002,-1.754e-05,-4.941e-05,-1.857e-06,-0.002,-0.006,-0.002,-5.787e-04,-0.004,2.116e-01,-0.001,-8.755e-06,-0.005,-2.400e-05
C40,-2.783e-04,-6.430e-06,-2.720e-04,-1.667e-04,-0.009,-0.002,-1.883e-06,-0.006,-0.002,-4.656e-04,-0.002,-0.003,-0.003,-4.090e-04,-2.814e-04,-1.053e-05,-6.098e-05,-1.665e-05,-5.051e-05,-3.676e-04,-3.206e-04,-1.100e-04,-1.701e-05,-0.003,-8.506e-05,...,-0.001,-0.002,-4.530e-05,-0.008,-3.753e-06,-6.783e-06,-1.124e-04,-0.009,-7.528e-04,-4.429e-04,-0.043,-0.002,-1.754e-05,-4.941e-05,-1.857e-06,-0.002,-0.006,-0.002,-5.787e-04,-0.004,-8.210e-05,0.520,-8.755e-06,-0.005,-2.400e-05
EX30,-2.783e-04,-6.430e-06,-2.720e-04,-1

# Cost

In [44]:
product_data['costs'] = BLP_results.compute_costs()
product_data['profit_per_car'] = product_data['prices'] - product_data['costs']
product_data['markups'] = product_data['profit_per_car'] / product_data['costs']
product_data['profits'] = BLP_results.compute_profits()
product_data[['prices', 'costs', 'profit_per_car', 'markups', 'profits']].describe()

,prices,costs,profit_per_car,markups,profits
count,334.000,334.000,334.000,334.000,3.340e+02
mean,46.911,130.133,-83.222,-0.665,-1.809e-01
std,29.975,30.272,1.233,0.115,5.098e-01
min,12.486,95.191,-92.772,-0.869,-8.177e+00
25%,28.398,111.138,-83.418,-0.744,-1.792e-01
50%,35.904,119.248,-82.794,-0.698,-5.952e-02
75%,52.962,137.283,-82.662,-0.614,-1.015e-02
max,194.052,277.612,-82.602,-0.301,-3.590e-04


In [45]:
product_data.loc[product_data['market_ids'] == 2023]

,ID,market_ids,shares,firm_ids,product_ids,Range,prices,HP,Chargetime,Type,Segment,Country,Sales,Intercept,demand_instruments0,demand_instruments1,demand_instruments2,China,outside_share,logit_delta,costs,profit_per_car,markups,profits
2,1,2023,8.690e-04,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1.0,41774,26403,3241,1,0.696,-6.686,109.127,-82.675,-0.758,-0.072
3,2,2023,1.473e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1.0,41769,26390,3241,1,0.696,-10.764,118.739,-82.675,-0.696,-0.001
8,3,2023,3.338e-04,Audi,e-tron,37.5,67.304,40.2,17,SUV,F,DE,68,1.0,41799,26202,3258,0,0.696,-7.643,151.094,-83.790,-0.555,-0.028
11,4,2023,1.129e-04,Audi,e-tron GT,47.2,121.934,52.2,17,Sedan,F,DE,23,1.0,41702,26082,3258,0,0.696,-8.727,205.724,-83.790,-0.407,-0.009
14,5,2023,1.153e-02,Audi,Q4 e-tron,49.6,65.855,28.1,28,SUV,C,DE,2349,1.0,41678,26323,3247,0,0.696,-4.100,149.645,-83.790,-0.560,-0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,187,2023,3.878e-04,Volkswagen,up!,25.6,17.485,8.1,48,Hatchback,A,DE,79,1.0,41918,26523,3227,0,0.696,-7.493,102.668,-85.182,-0.830,-0.033
328,188,2023,2.538e-03,Volvo,C40,46.6,43.066,40.2,28,SUV,C,SE,517,1.0,41708,26202,3247,0,0.696,-5.614,126.689,-83.622,-0.660,-0.212
329,189,2023,1.964e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,1.0,41699,26336,3247,0,0.696,-10.476,120.447,-83.622,-0.694,-0.002
332,190,2023,9.647e-03,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,1.0,41717,26202,3247,0,0.696,-4.279,127.549,-83.622,-0.656,-0.807


In [46]:
statsmodels_ols = smf.ols('np.log(costs) ~ 1 + Range + HP + Chargetime', product_data)
statsmodels_results = statsmodels_ols.fit(cov_type='HC0')
statsmodels_results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.629    
Dependent Variable: np.log(costs)    AIC:                -453.9122
Date:               2024-06-03 22:36 BIC:                -438.6676
No. Observations:   334              Log-Likelihood:     230.96   
Df Model:           3                F-statistic:        148.5    
Df Residuals:       330              Prob (F-statistic): 6.61e-61 
R-squared:          0.633            Scale:              0.014864 
-------------------------------------------------------------------
               Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
-------------------------------------------------------------------
Intercept      4.6204    0.0378  122.2682  0.0000   4.5463   4.6944
Range          0.0030    0.0011    2.6468  0.0081   0.0008   0.0052
HP             0.0076    0.0006   11.9383  0.0000   0.0064   0.0089
Chargetime    -0.0025    0.0005   -4.6577  0.0000  -0.0035  -0.0014
------------------------------------------------------------------
Omnibus:             123.931       Durbin-Watson:          0.601  
Prob(Omnibus):       0.000         Jarque-Bera (JB):       441.283
Skew:                1.626         Prob(JB):               0.000  
Kurtosis:            7.597         Condition No.:          333    
==================================================================

"""

In [50]:
eq_prices = BLP_results.compute_prices()
product_data['eq_prices'] = eq_prices
prices = pd.DataFrame(eq_prices, index = product_data['product_ids'])

In [51]:
prices.loc['Model Y']

,0
product_ids,
Model Y,50.324
Model Y,52.115
Model Y,40.555
